rewrite a ann_text which return dep_list I can direct put into graph

In [1]:
import spacy
import neuralcoref
from spacy import displacy # https://spacy.io/usage/visualizers
def show_depparse(text):
    doc = nlp(text)
    displacy.render(doc, style='dep',jupyter=True)

nlp = spacy.load("en")

##### remove_pronoun

In [2]:
def remove_pronoun(input_t): # replace pronoun with original noun
    neuralcoref.add_to_pipe(nlp)
    doc = nlp(input_t)
    cvt_dict = dict()
    for s in doc._.coref_clusters: # find pronouns save it as dict
        cvt_dict[str(s.main)] = [str(i) for i in s.mentions if i != s.main]
#    print("pronoun dict:", cvt_dict)
    nlp.remove_pipe("neuralcoref")
    
    pn_list = [] # save list as (pronoun, origin noun)
    for pn_main in cvt_dict:
        for pn in cvt_dict[pn_main]:
            pn_list.append( (pn, pn_main) )
    rm_dup_pronouns(pn_list)
#    print("pronoun list without dup:", pn_list)
    
    no_pronoun_t = [] # replace pronoun with origin noun
    for tok in doc:
        if tok.lemma_ == "-PRON-":
            replace_pronoun = False
            for pn_idx, pn in enumerate(pn_list):
                if tok.text == pn[0]:
                    replace_pronoun = True
                    no_pronoun_t.append(pn[1])
                    del(pn_list[pn_idx])
            if pn_list is None or replace_pronoun is False:
                no_pronoun_t.append(tok.text)
        else:
            no_pronoun_t.append(tok.text)
    new_t = " ".join(no_pronoun_t)

    return new_t

def rm_dup_pronouns(pn_list):
    rm_idx_list = []
    for idx, i in enumerate(pn_list):
        if i[0] == i[1]:
            rm_idx_list.append(idx)
    rm_idx_list.reverse()
    for rm_idx in rm_idx_list:
        del pn_list[rm_idx]
    # # example
    # pn_list = [('his', 'Lo Shang-hua (羅尚樺)'),("it","it"), ("the","the"), ('he', 'Lo Shang-hua (羅尚樺)'), ('Lo', 'Lo Shang-hua (羅尚樺)'), ('his aircraft', 'his aircraft'), ('his', 'Pan’s'), ('He', 'Pan’s'), ('his', 'Pan’s'), ('it', 'The Taitung District Prosecutors’ Office'), ('its', 'The Taitung District Prosecutors’ Office')]
    # rm_dup_pronouns(pn_list)
    # pn_list
    
t = 'Angela lives in Boston. She is quite happy in that city. She enjoy her life.'
#t = "He is nice."
remove_pronoun(t)

'Angela lives in Boston . Angela is quite happy in that city . Angela enjoy Angela life .'

##### dep_create

###### dep relation format

In [123]:
def ann_text(text):
    no_pronoun_text = remove_pronoun(text)
    doc = nlp(no_pronoun_text)
    
    dep_list = []
    for sent_idx, sent in enumerate(doc.sents):
        for tok_idx, tok in enumerate(sent):
            dep = find_dep(tok)
            if dep is not None:
                dep_list.extend( dep )
        print("===== next semt =====")
    return dep_list

def dep_fwfw(tok, dp1, dp2, double_dir=False, reverse=False): # dep_forward_forward
    # A->B->C  => C->A
    #     ex: table for food
    #         dep = (table-->for), (for-->food)
    #             => table<--food
    source = None
    target = None
    for child in tok.children:
        if child.dep_ == dp1:
            for cchild in child.children:
                if cchild.dep_ == dp2:
                    source = tok
                    target = cchild
    return creat_dep(source, target, double_dir, reverse)

def dep_side(tok, dp1, dp2, double_dir=False, reverse=False):
    # A<-B->C => C->A
    #     ex: He is nice.   dep = (is->he), (is->nice) 
    #                           => nice->he
    source = None
    target = None
    for child in tok.children:
        if child.dep_ == dp1:
            target = child
        if child.dep_ == dp2:
            source = child
    return creat_dep(source, target, double_dir, reverse)

# def dep_chg(tok, dp, double_dir=False, reverse=True): # dep_change
#     # A->B => B->A
#     #     ex: poor cat     dep = (cat->poor)
#     #                          => poor->cat
#     source = None
#     target = None
#     for child in tok.children:
#         if child.dep_ == dp:
#             source = child
#             target = tok
#     return creat_dep(source, target, double_dir, reverse)

def dep_rem(tok, dp, double_dir=False, reverse=False): # dep_remain
    # A->B => A->B
    #     ex: find->it     dep = (find->it)
    #                          => find->it
    source = None
    target = None
    for child in tok.children:
        if child.dep_ == dp:
            source = tok
            target = child
    return creat_dep(source, target, double_dir, reverse)
    
    
def creat_dep(source, target, double_dir=False, reverse=False):
    if source is not None and target is not None:
        if double_dir:
            return [( tok2node(source), tok2node(target) ), 
                    ( tok2node(target), tok2node(source) )]
        else:
            if reverse: # source -> target
                return (tok2node(target), tok2node(source) )
            else:
                return (tok2node(source), tok2node(target) )
    else:
        return None
def tok2node(token): # token => str( lemma=pos )
    tok_lemma = token.lemma_
    if token.lemma_ == "-PRON-":
        tok_lemma = token.text.lower()
    if token.pos_ == "PUNCT":
        return token.text
    else:
        return "".join([tok_lemma, '=', token.pos_])


###### creat dep

In [226]:
def find_dep(tok):
    tmp_dep_list = []
    tmp_dep_list.append( nsubj(tok) )
    tmp_dep_list.append( nsubjp(tok))
    tmp_dep_list.append( same(tok)  )
    tmp_dep_list.append( ppobj(tok) )
    tmp_dep_list.append( agpbj(tok) )
    tmp_dep_list.append( dative(tok))
    tmp_dep_list.append( ppcomp(tok))
    tmp_dep_list.append( advcl(tok) )
    tmp_dep_list.append(npadvmod(tok))
    tmp_dep_list.append( amod(tok)  )
    tmp_dep_list.append( advmod(tok))
    tmp_dep_list.append( dobj(tok)  )
    tmp_dep_list.append( nummod(tok))
    tmp_dep_list.append( xcomp(tok) )
    tmp_dep_list.append( ccomp(tok) )
    tmp_dep_list.append( acl(tok)   )
    tmp_dep_list.append( poss(tok)  )
    tmp_dep_list.append( relcl(tok) )
    tmp_dep_list.append( oprd(tok)  )
    return flat_rmNone_list(tmp_dep_list)

def flat_rmNone_list(tmp_dep_list):
    dep_list = []
    for dep in tmp_dep_list:
        if dep is None:
            continue
        elif type(dep) is list:
            dep_list.extend(dep)
        else:
            dep_list.append(dep)
    return dep_list

def nsubj(tok):
    dep = dep_side(tok, 'nsubj', 'acomp') # "I am nice"
    if dep is None:
        dep = dep_side(tok, 'nsubj', 'attr', double_dir=True)  #"Tom is man"
    if dep is None:
        dep = dep_rem(tok, 'nsubj') # "I ran home"
    return dep
def nsubjp(tok):
    return dep_rem(tok, 'nsubjpass') # "dog was found"
def same(tok):
    return dep_rem(tok, 'appos', double_dir=True) # "Sam, the VIP"
def agpbj(tok): # agent+pobj
    return dep_fwfw(tok, 'agent', 'pobj') # "taken by us"
def ppobj(tok): # prep+pobj
    if tok.pos_ in ['NOUN', 'PROPN']: 
        return dep_fwfw(tok, 'prep', 'pobj', reverse=True) # "table of picnic"
    else:                             
        return dep_fwfw(tok, 'prep', 'pobj') # "some of toys"
def dative(tok):
    return dep_rem(tok, 'dative') # "gave me book"
def ppcomp(tok):
    return dep_fwfw(tok, 'prep', 'pcomp') # "play at flying"
def advcl(tok):
    return dep_rem(tok, 'advcl') #"cry when fail"
def npadvmod(tok):
    return dep_rem(tok, 'npadvmod', reverse=False) # "done this morning"
def amod(tok):
    return dep_rem(tok, 'amod', reverse=True) # "poor student"
def advmod(tok):
    return dep_rem(tok, 'advmod', reverse=True) # "less often"
def dobj(tok):
    return dep_rem(tok, 'dobj') # "find it"
def nummod(tok):
    return dep_rem(tok, 'nummod', reverse=True) # "ten books"
def xcomp(tok):
    return dep_rem(tok, "xcomp") # "easy to play"
def ccomp(tok):
    return dep_rem(tok, "ccomp") # "I conside him fool"
def poss(tok):
    return dep_rem(tok, "poss", reverse=True) # "his gun"
def relcl(tok):
    return dep_rem(tok, "relcl", reverse=True) # "girl who likes me"
def oprd(tok):
    return dep_rem(tok, "oprd") # "made public"
def acl(tok):
    return dep_rem(tok, "acl", reverse=True) # "fact that nobody care"

ann_text(t)

===== next semt =====


[('make=VERB', 'public=ADJ')]

In [40]:
t = "“Since many of the things found on the body belonged to Pan Ying-chun, it is probably him,” Pan’s brother told reporters at the morgue yesterday. Finding the body was enough, he said, adding that he just wants to give his brother a proper funeral and send him off."

In [72]:
def tok2node(token): # token => str( lemma=pos )
    tok_lemma = token.lemma_
    if token.lemma_ == "-PRON-":
        tok_lemma = token.text.lower()
    if token.pos_ == "PUNCT":
        return token.text
    else:
        return "".join([tok_lemma, '=', token.pos_])

#text = 'Angela lives in Boston Big Set (BBS), she is quite happy in that city. She enjoy her life.'
text = "He is nice."

no_pronoun_text = remove_pronoun(text)
doc = nlp(no_pronoun_text)
for sent_idx, sent in enumerate(doc.sents):
    for tok in sent:
        print(tok2node(tok))

he=PRON
be=VERB
nice=ADJ
.


In [204]:
t = "Chin Ting, Kuo"
show_depparse(t)